# 图像转换

Opencv主要提供了两个转换函数 warpAffine 和 warpPerspective完成全部的转换的工作


## 仿射变换（Affine transformation）
在讨论仿射变换之前，我们先了解一下欧几里得变换（Euclidean transformation）。欧几里得变换是一种几何变换，任何对象在进行欧几里得变换后会保持原来的形状和大小，严格一些说原空间的任何两个点，在经过转换后，它们的距离不变。欧几里得变换包括旋转，位移，反射以及它们的组合。如何在欧几里得变换上附加偏手性（handedness），反射就不是欧几里得变换了，因为右手在转换后会变成左手。

仿射变换是欧几里得变换的扩展，原空间中的点，直线和面在转换后仍然得以保持，同时它还保持线的并行性。但是其不保持相交线的角度和两个点的距离。仿射变换保持位移（translation），缩放（scaling），反射（reflection），旋转（rotation），错切（shear）等。


### 位移（translation）

移动使用如下的转换矩阵

$ T = \begin{bmatrix}
        1 & 0 & t_x \\
        0 & 1 & t_y 
        \end{bmatrix} $

In [ ]:
import cv2
import numpy as np

读入Image文件

In [ ]:
img = cv2.imread('/Users/davidyon/Desktop/IMG_9261.JPG')

获取Image文件的维度信息

In [ ]:
num_rows,num_cols = img.shape[:2]

设置转换矩阵，向右移动350（tx），向下移动550（ty），对应的矩阵如下：

In [ ]:
move_matrix = np.float32([ [1,0,350], [0,1,550] ])

使用函数 cv2.warpAffine 进行转换，这个函数使用 2x3 的转换矩阵作为输入。另外一个转换函数 cv.warpPerspective 使用 3x3 的转换矩阵作为输入

In [ ]:
img_move = cv2.warpAffine(img, move_matrix, (num_cols+700,num_rows+1100), cv2.INTER_LINEAR, cv2.BORDER_WRAP, 2)

其中，(num_cols+700,num_rows+1100) 是转换后的图像维度，转换的图像宽度增加700，高度增加1100，这样可以使移动（350，550）后的图像显示在正中间。后面两个参数用于指定如何填充移动的边框。

In [ ]:
cv2.imshow('Translation_Move', img_move)
cv2.waitKey()
cv2.destroyAllWindows()
for i in range(1,5):
    cv2.waitKey(1)

之所以后面有循环，是因为在MacOS上destroyAllWindows不起作用，但是增加几个waitKey调用就可以删除窗口

###  旋转 （rotate）
图像旋转使用如下的转化矩阵

$
R=\begin{bmatrix}
    \cos \theta  & -\sin \theta \\
    \sin \theta  & \cos \theta
  \end{bmatrix}
$

其中 $ \theta $ 是逆时针旋转的角度。OpenCV 的函数 `getRotationMatrix2D` 提供针对旋转矩阵的细粒度的控制。我们可以指定图像基于那个点进行旋转，旋转的角度，以及对图像的缩放等参数。一旦我们定义了这个旋转矩阵，我们可以使用 `warpAffine` 函数将这个矩阵应用到任何图像。

In [ ]:
import cv2
import numpy as np
img = cv2.imread('/Users/davidyon/Desktop/IMG_9261.JPG')
num_rows, num_cols = img.shape[:2]
translation_matrix = np.float32([ [1,0,int(0.5*num_cols)], [0,1,int(0.5*num_rows)] ])
rotation_matrix = cv2.getRotationMatrix2D((num_cols, num_rows),30,1)
img_translation = cv2.warpAffine(img, translation_matrix, (2*num_cols, 2*num_rows))
img_rotation = cv2.warpAffine(img_translation,rotation_matrix,(2*num_cols, 2*num_rows))
cv2.imshow('Rotation',img_rotation)
cv2.waitKey()
cv2.destroyAllWindows()
cv2.waitKey(1)

其中，下面的代码是为了避免旋转后有些部分被截掉而增加显示的空间，并将图像向中间移动. 

```python
translation_matrix = np.float32([ [1,0,int(0.5*num_cols)], [0,1,int(0.5*num_rows)] ])
...
img_translation = cv2.warpAffine(img, translation_matrix, (2*num_cols, 2*num_rows))
```

### 缩放（scale）
OpenCV 提供 `resize` 函数对图像进行缩放，并提供相应的缩放参数

In [ ]:
import cv2
img = cv2.imread('/Users/davidyon/Desktop/IMG_9261.JPG')
img_scaled = cv2.resize(img,None,fx=1.2, fy=1.2, interpolation =
                        cv2.INTER_LINEAR)
cv2.imshow('Scaling - Linear Interpolation', img_scaled)
img_scaled = cv2.resize(img,None,fx=1.2, fy=1.2, interpolation =
   cv2.INTER_CUBIC)
cv2.imshow('Scaling - Cubic Interpolation', img_scaled)
img_scaled = cv2.resize(img,(450, 400), interpolation = cv2.INTER_AREA)
cv2.imshow('Scaling - Skewed Size', img_scaled)
cv2.waitKey(5000)
cv2.destroyAllWindows()
cv2.waitKey(1)

### 错切（shear）
要构造一个通用的仿射变换矩阵，我们需要定义控制点。确定了这些控制点后，我们需要确定要把这些控制点映射到何处。对于仿射变换，我们需要做的是在原来的图像中找出三个点，然后确定这三个点在目标图像中的三个位置。下面，我们通过定义原图像中的三个顶点和其相应的目标图像中的三个映射点完成一个错切变换。

In [ ]:
import cv2
import numpy as np
img = cv2.imread("/Users/davidyon/Desktop/IMG_9261.JPG")
rows, cols = img.shape[:2]
src_points = np.float32([[0,0], [cols-1,0], [0,rows-1]])
dst_points = np.float32([[0,0], [int(0.6*(cols-1)),0],
[int(0.4*(cols-1)),rows-1]])
affine_matrix = cv2.getAffineTransform(src_points, dst_points)
img_output = cv2.warpAffine(img, affine_matrix, (cols,rows))
print(img_output.shape)
cv2.imshow('Input', img)
cv2.imshow('Output', img_output)
cv2.waitKey()
cv2.destroyAllWindows()
cv2.waitKey()

### 反射（reflect）
同样我们可以通过这种方式完成反射变换。与上面的代码相比，唯一的不同点是定义的三个目标点不同


In [ ]:
import cv2
import numpy as np
img = cv2.imread("/Users/davidyon/Desktop/IMG_9261.JPG")
rows, cols = img.shape[:2]
src_points = np.float32([[0,0], [cols-1,0], [0,rows-1]])
dst_points = np.float32([[cols-1,0], [0,0], [cols-1,rows-1]])
affine_matrix = cv2.getAffineTransform(src_points, dst_points)
img_output = cv2.warpAffine(img, affine_matrix, (cols,rows))
print(img_output.shape)
cv2.imshow('Input', img)
cv2.imshow('Output', img_output)
cv2.waitKey()
cv2.destroyAllWindows()
cv2.waitKey()

(3744, 5616, 3)
